# 朴素贝叶斯

In [27]:
# 导入必要的库
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
import time
import numpy as np


## 加载数据集，划分训练集和验证集

In [28]:
# 加载20newsgroups数据集
print("正在加载20newsgroups数据集...")

# 加载完整数据集,remove的作用是去除数据集中的头部、尾部和引号,这里不要remove，因为头尾都可能包含重要信息
newsgroups = fetch_20newsgroups(data_home='./data', subset='all', shuffle=True, random_state=42)
"""
subset='all'  'train'  'test'  指定加载哪一部分数据
shuffle=True : 是否在加载数据时将其打乱顺序。
"""

# 分割为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(newsgroups.data, newsgroups.target, test_size=0.2, random_state=42)


# 查看数据集基本信息
print(f"训练集大小: {len(X_train)}")
print(f"测试集大小: {len(X_test)}")
print(f"类别（标签）数量: {len(newsgroups.target_names)}")
print('前50个样本的类别(标签): ',y_train[:50])
print(f"类别（标签）名称: {newsgroups.target_names}")



#  数据预处理，创建适合文本分类的 TF-IDF向量化器
print("\nIF-IDF适合对文本分类,创建TF-IDF向量化器...")
tfidf = TfidfVectorizer(stop_words='english')       
"""
超参数：
max_features :特征的最大数量。 特征越多，计算量越大。（这里的特征就是分词）
stop_words   :停用词。        停用词也可以减少特征
"""

# 划分训练集和测试集并进行TF-IDF向量化


X_train_tfidf = tfidf.fit_transform(X_train)
print(f'X_train_tfidf.shape:  {X_train_tfidf.shape} : (样本数，特征数)')
X_test_tfidf = tfidf.transform(X_test)

正在加载20newsgroups数据集...
训练集大小: 15076
测试集大小: 3770
类别（标签）数量: 20
前50个样本的类别(标签):  [ 5  8 13  0  9 15 13 18 16 13  3  4 11 11 14  9 17 14  0  3 15  0  4  4
 12 18 16  4 15 13  0  5  8 18  6  9  9 17 15 12 17 16 15  7 14 12 13 11
  9 19]
类别（标签）名称: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

IF-IDF适合对文本分类,创建TF-IDF向量化器...
X_train_tfidf.shape:  (15076, 148992) : (样本数，特征数)


## 训练模型

In [29]:
# 创建并训练朴素贝叶斯分类器
print("正在训练模型...")
clf = MultinomialNB(alpha=0.1)
"""超参数
拉普拉斯平滑:  alpha=0.1
"""

# 训练模型
clf.fit(X_train_tfidf, y_train)


正在训练模型...


,alpha,0.1
,force_alpha,True
,fit_prior,True
,class_prior,None


## 评估模型

In [30]:
# 在测试集上进行预测
print("在测试集上进行预测...")
y_pred = clf.predict(X_test_tfidf)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f"\n测试集准确率: {accuracy:.4f}")


# 计算精确率
precision = precision_score(y_test, y_pred, average='weighted')
print(f"精确率 (Precision): {precision:.4f}")

# 计算召回率
recall = recall_score(y_test, y_pred, average='weighted')
print(f"召回率 (Recall): {recall:.4f}")

# 计算AUC 多分类 采用multi_class='ovr'
# predict_proba方法会输出每个样本属于各个类别的概率（proba即probability的缩写）
y_proba = clf.predict_proba(X_test_tfidf)
auc = roc_auc_score(y_test, y_proba, multi_class='ovr', average='weighted')
print(f"AUC: {auc:.4f}")



在测试集上进行预测...

测试集准确率: 0.9098
精确率 (Precision): 0.9131
召回率 (Recall): 0.9098
AUC: 0.9960
